# Metryki klasyfikacji

Aby poznać popularne metody oceny klasyfikatorów wygenerujemy sobie przykładowe sekwencje klas przykładów. Wykorzystamy w tym celu bibliotekę numpy oraz generator liczb losowych z rozkładu normalnego.

In [3]:
import numpy as np
from numpy.random import RandomState

In [4]:
random = RandomState(30)

random_1 = random.normal(loc = 0.0, size = 100)
random_2 = random.logistic(loc = 1, size = 100)

In [5]:
print(random_1[:10])
print(random_2[:10])

[-1.26405266  1.52790535 -0.97071094  0.47055962 -0.10069672  0.30379318
 -1.72596243  1.58509537  0.13429659 -1.10685547]
[-0.34625296  2.7465497   0.46063247  3.2000744   0.12353606  1.56724961
  2.32137633 -0.21323077  1.9572528   0.03456367]


In [6]:
y_test = [1 if i >= 0 else 0 for i in random_1]
y_pred = [1 if i >= 0 else 0 for i in random_2]

In [7]:
print(y_test[:10])
print(y_pred[:10])

[0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]


## Macierz błędów

Pierwszym krokiem będzie stworzenie macierzy błędów, czyli wyznaczenia true postives, true negatives, false positives i false negatives.

In [8]:
def confusion_matrix(truth, prediction):
    tp, tn, fp, fn = 0, 0, 0, 0
    
    for label_t, label_p in zip(truth, prediction):
        if label_t == label_p:
            if label_p == 1:
                tp += 1
            else:
                tn += 1
        else:
            if label_p == 1:
                fp += 1
            else:
                fn +=1
    return tp, tn, fp, fn

In [9]:
tp, tn, fp, fn = confusion_matrix(y_test, y_pred)

In [10]:
print(f'TP:{tp}, TN:{tn}')
print(f'Ile obserwacji zaklasyfikowaliśmy poprawnie: {tp+tn}')

TP:30, TN:14
Ile obserwacji zaklasyfikowaliśmy poprawnie: 44


In [11]:
print(f'FP:{fp}, FN:{fn}')
print(f'Ile obserwacji zaklasyfikowaliśmy niepoprawnie: {fp+fn}')

FP:43, FN:13
Ile obserwacji zaklasyfikowaliśmy niepoprawnie: 56


## Accuracy

Pierwszą miarą jest miara dokładności 

accuracy = (tp+tn) / (tp+tn+fp+fn)

In [12]:
def accuracy(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    print(tp, tn, fp, fn)
    
    return (tp+tn) / (tp+tn+fp+fn)

Wynik dla analizowanych predykcji

In [13]:
acc = accuracy(y_test, y_pred)

print(f'Dokładność: {acc}')

30 14 43 13
Dokładność: 0.44


#### Problem
Accuracy zachowuje się źle przy niezbalansowanych zbiorach. Wygenerujemy zbiór, w którym większość przykładów będzie negatywna.

In [14]:
random_3 = random.normal(loc = -1.5, size = 100)
y_pred_2 = [1 if i >= 0 else 0 for i in random_3]

In [15]:
sum(y_pred_2)

8

In [16]:
accuracy(y_test, y_pred_2)

3 52 5 40


0.55

Zakładając z góry, że wszystkie przykłady są negatywne uzyskujemy bardzo wysoką dokładność. Takie zachowanie jest w wielu przypadkach niepożądane - przypuśćmy, że próbujemy stworzyć klasyfikator do komórek rakowych, gdzie większość przypadków jest negatywna - dla takiego podejścia, zwrócenie informacji że wszystkie przypadki są negatywne da bardzo wysoką dokładność.

In [17]:
accuracy(100*[0], y_pred_2)

0 92 8 0


0.92

In [18]:
y_test[:10]

[0, 1, 0, 1, 0, 1, 0, 1, 1, 0]

In [19]:
y_pred[:10]

[0, 1, 1, 1, 1, 1, 1, 0, 1, 1]

In [20]:
y_pred_2[:10]

[0, 1, 0, 0, 0, 0, 1, 0, 1, 0]

## Recall

Ta miara z kolei mówi o tym, jaką część dodatnich wyników wykrył klasyfikator.

recall = tp / (tp + fn)

In [21]:
def recall(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    
    return tp / (tp+fn)

In [22]:
recall(y_test, y_pred)

0.6976744186046512

In [23]:
recall(y_test, y_pred_2)

0.06976744186046512

## Precision

Jest to miara, która skupia się tylko na przykładach pozytywnych - mówi jaka część wyników wskazanych przez klasyfikator jako dodatnie jest rzeczywiście dodatnia.

precision = tp / (tp+fp)

In [24]:
def precision(truth, prediction):
    tp, tn, fp, fn = confusion_matrix(truth, prediction)
    
    return tp / (tp+fp)

In [25]:
precision(y_test, y_pred)

0.410958904109589

In [26]:
precision(y_test, y_pred_2)

0.375

# F1 Score

Jest to średnia harmoniczna precyzji i czułości. Ogólnie - im wyższy F1-score tym lepszy jest klasyfikator.

f_score = (2 * precision * recall)/(prec+rec)

In [27]:
def f1_score(truth, predicted):
    prec = precision(truth, predicted)
    rec = recall(truth, predicted)
    
    return 2*prec*rec / (prec*rec)

In [28]:
f1_score(y_test, y_pred)

2.0

In [29]:
f1_score(y_test, y_pred_2)

2.0

**Zadanie:** Sprawdź funckje f1_score, precision_score, recall_score z modułu scikit learn

In [41]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

In [42]:
recall_score(y_test, y_pred)

0.6976744186046512

In [43]:
precision_score(y_test, y_pred)

0.410958904109589

In [44]:
f1_score(y_test, y_pred)

0.5172413793103448

In [45]:
confusion_matrix(y_test, y_pred)

array([[14, 43],
       [13, 30]])

In [46]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.52      0.25      0.33        57
           1       0.41      0.70      0.52        43

    accuracy                           0.44       100
   macro avg       0.46      0.47      0.43       100
weighted avg       0.47      0.44      0.41       100

